### Connection  to Drive and Importing libraries

In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive/UNIVERSITA/ANN/ANNDL/Homework1



Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/UNIVERSITA/ANN/ANNDL/Homework1


NameError: ignored

In [5]:

# from cartella.file import function_name
from function_folder.librarys import librarys_upload
librarys_upload()

42


### Load and process the dataset

In [ ]:
from function_folder.librarys import librarys_upload

In [ ]:
# Conditional check for unzipping
unzip = False

# Unzip the 'animals.zip' file if the 'unzip' flag is True
if unzip:
    !unzip public_data.zip


from function_folder.load_images_from_folder import load_images_from_folder
items_path = 'public_data/'
items = load_images_from_folder(items_path)

### Splitting the dataset

In [ ]:
from function_folder.splitter import splitting_function
X_train, X_val,X_test, y_train, y_val,y_test = splitting_function(X,y)

## Model

In [ ]:
from function_folder.models.model0 import MobileNetV2
mobile = MobileNetV2()

# Display the model architecture with input shapes
tfk.utils.plot_model(mobile, show_shapes=True)

In [ ]:
# Use the supernet as feature extractor, i.e. freeze all its weigths
mobile.trainable = False

# Create an input layer with shape (224, 224, 3)
inputs = tfk.Input(shape=(224, 224, 3))
# Connect MobileNetV2 to the input
x = mobile(inputs)
# Add a Dense layer with 2 units and softmax activation as the classifier
outputs = tfkl.Dense(2, activation='softmax')(x)

# Create a Model connecting input and output
tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model with Categorical Cross-Entropy loss and Adam optimizer
tl_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics=['accuracy'])

# Display model summary
tl_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 mobilenetv2_1.00_224 (Func  (None, 1280)              2257984   
 tional)                                                         
                                                                 
 dense (Dense)               (None, 2)                 2562      
                                                                 
Total params: 2260546 (8.62 MB)
Trainable params: 2226434 (8.49 MB)
Non-trainable params: 34112 (133.25 KB)
_________________________________________________________________


### Train the model

In [ ]:
# Train the model
tl_history = tl_model.fit(
    x = preprocess_input(X_train*255), # We need to apply the preprocessing thought for the MobileNetV2 network
    y = y_train,
    batch_size = 16,
    epochs = 200,
    validation_data = (preprocess_input(X_val*255), y_val), # We need to apply the preprocessing thought for the MobileNetV2 network
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=20, restore_best_weights=True)]
).history

### Plotting

In [ ]:
from function_folder.plotting.history_plot import plot_history

plot_history(tl_history)

## Save the model

In [ ]:
# Save the best model
tl_model.save('Model_0')
del tl_model

## Fine Tuning

In [ ]:
# Re-load the model after transfer learning
ft_model = tfk.models.load_model('TransferLearningModel')
ft_model.summary()

# Set all MobileNetV2 layers as trainable
ft_model.get_layer('mobilenetv2_1.00_224').trainable = True
for i, layer in enumerate(ft_model.get_layer('mobilenetv2_1.00_224').layers):
   print(i, layer.name, layer.trainable)


# Freeze first N layers, e.g., until the 133rd one
N = 133
for i, layer in enumerate(ft_model.get_layer('mobilenetv2_1.00_224').layers[:N]):
  layer.trainable=False
for i, layer in enumerate(ft_model.get_layer('mobilenetv2_1.00_224').layers):
   print(i, layer.name, layer.trainable)
ft_model.summary()


# Compile the model
ft_model.compile(loss=tfk.losses.BinaryCrossentropy(), optimizer=tfk.optimizers.Adam(1e-5), metrics='accuracy')

In [ ]:
# Fine-tune the model
ft_history = ft_model.fit(
    x = preprocess_input(X_train*255), # We need to apply the preprocessing thought for the MobileNetV2 network
    y = y_train,
    batch_size = 16,
    epochs = 200,
    validation_data = (preprocess_input(X_val*255), y_val), # We need to apply the preprocessing thought for the MobileNetV2 network
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=20, restore_best_weights=True)]
).history

## Plot

In [ ]:
plot_history(ft_history)

### Evaluation on test set

In [ ]:
# Evaluate the model on the test set
test_accuracy = ft_model.evaluate(preprocess_input(X_test*255),y_test,verbose=0)[-1]
print('Test set accuracy %.4f' % test_accuracy)